# Introduction to Spectral Analysis I

The purpose of spectral analysis of time series is to decompose the temporal behavior into a set of waves or cycles with characteristic lengths (or frequencies) and estimate the variance that they contribute to the total variance of the time series.  This allows us to identify cyclical components from observations that may provide insight into underlying causality or form the basis for predictability and forecasting.  Spectral analysis methods are primarily concerned with moving temporal data (attached to the dimension of time) into frequecny space (attached to the dimension that is the frequency of wavelength of the cycles we identify) while making various trade-offs that are necesaary because of the finite and noisy nature of our time series observations. 

As with the other topics in this class, one could teach an entire course simply on spectral analysis methods, applications, and cautions.  This and the following notebooks will provide a brief introduction to a few of the methods and some of the things you should look out for when doing your own spectral analysis.  Here are a few additional resources:

- Stoica and Moses's [_Spectral Analysis of Signals_](https://www.maths.lu.se/fileadmin/maths/personal_staff/Andreas_Jakobsson/StoicaM05.pdf)
- [Spectral Analysis of Paleoclimate Data](https://comptools.climatematch.io/tutorials/W1D4_Paleoclimate/student/W1D4_Tutorial6.html)
- [Spectral Analysis with Pyleoclim](https://linked.earth/PyleoTutorials/notebooks/L2_spectral_analysis.html)
- Scipy's [signal module for spectral analysis](https://docs.scipy.org/doc/scipy/reference/signal.html#spectral-analysis)

As we begin, this quote from [Muller and MacDonald's 2002 book on astronomical causes of the ice ages](https://link.springer.com/book/9783540437796) is useful to keep in mind:

> Although there are many ways to do an incorrect spectral analysis, there is no "correct way” … Be aware that all approaches to spectral analysis entail assumptions, and that the differences in these assumptions often account for the different results of the analyses. If you know what those assumptions are, you can make a reasoned judgement of what conclusions you should draw.

Let's get our libraries.  Modules for spectral analysis are in various libraries across the Python ecosystem, but we'll focus mostly on those in Scipy and Numpy for now.  In the next class we'll introduce the Multitaper method (MTM), and make use of some different libraries for this. 

In [1]:
import math
import numpy as np 
import scipy as sp
import pandas as pd 

 # gets the signal processing capability of SciPy
from scipy import signal

# statsmodels has a simple Autocorrelation function we can use for a demonstration of the link between cycles and autocorrelation 
from statsmodels.tsa.stattools import acf

# Matplotlib plotting functionality as well as a helper for axes formatting
import matplotlib.pyplot as plt # this line pulls out just the part of matplotlib for creating plots, for simplicity
from matplotlib.ticker import ScalarFormatter # this will help us change the axes a bit easier

# you can omit the line below if you'd like, but I really don't like the default fonts in Python, so I switch to Helvetica
plt.rcParams['font.family'] = 'Helvetica'

# ignore some warnings for lecture, but recommend YOU only do this once you know what those warnings are! 
import warnings
warnings.filterwarnings("ignore")

Let's start by creating a simple periodic time series -- a sine wave.  The code in the block below does that for you.  Try and change the variable `cycles_per_time` to some different values, and observe how the periodic series changes.   

In [ ]:
# Set random seed for reproducibility
np.random.seed(1999)

# we'll use 128 time points again
n_samples = 128

# Create our time vector - e.g. 128 years long
t = np.arange(1, n_samples+1)

# Create a sine wave with frequency set to the variable cycles_per_time -- you can vary this - try 1, 2, 3, 4, or 5 here
cycles_per_time = 2
cycle_amplitude = 3 # change this to change the amplitude - power or variance - of the cyclical time series
u = 2 * np.pi * (cycles_per_time/128) * t
st = cycle_amplitude * np.sin(u)
    
# Plot the sine wave and observe how many cycles are completed per 128 'years'
plt.figure()
plt.plot(t, st, 'r', label='sin')
plt.xlabel('Years')
plt.xlim([1, 128])
plt.tight_layout()


Let's add some noise to this time series and plot again.  You can change the amplitude of the noise using variable `noise_amplitude` in the code below:

In [ ]:
# create a time series that is the sum of the sine wave above and some Gaussian noise with an amplitude of 'noise_amplitude' - you can see how this amplitude affects the spectrum below
noise_amplitude = 5
my_series = st + (noise_amplitude * np.random.randn(len(t))) # reproducible with random seed above

# Plot sum of signals
plt.figure()
plt.plot(t, st, 'k', label='Signal')
plt.plot(t, my_series, 'r-', label='Signal with noise')
plt.xlim([1, 128])
plt.xlabel('TIME')
plt.legend()

plt.show()

It is worth thinking about how the presence of cycles in the data is connected to autocorrelation (which we've discussed in the context of both correlation and time series analysis).  

In [4]:
lags = 40  # Number of lags for the autocorrelation 
autocorr0 = acf(my_series, nlags=lags)


We can plot the autocorrelation function for the pure cycle (and the cycle+noise case if we'd like).  The presense of a cycle creates long-lag autocorrelation in a time series, since every value is strongly related to the previous value in the series by the cyclical structure created by the sine function.  We'll leverage this self-similarity later when we look at alternative approaches to spectral analysis. 

In [ ]:
# Plot the autocorrelation function for the pure cycle
plt.stem(np.arange(lags + 1), autocorr0)
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.xticks(np.arange(0, lags + 1, 5))

plt.tight_layout()
plt.show()

We'll now use one of the simplest methods for moving from temporal space to frequency space and measuring the relative contribution of cyclical components with different wavelenghts: the periodogram.  While it has fallen out of favor for spectral analysis (it is considered a 'inconsistent estimator' of the population spectrum), but it is a good place to start and often can still be used to provide insights -- albiet sometimes rough estimates -- of the spectrum of the time series. 

We'll use Scipy's `periodogram` for this.  You can find more details [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.periodogram.html). 

In [6]:
# get the periodogram function from Scipy's signal module
from scipy.signal import periodogram 


The `periodogram` function accepts our data (arrays, DataSeries, etc.), as well as a number of options. One things we can do is pass the function our choice of [windows](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.get_window.html#scipy.signal.get_window) that can improve the spectral estimates (we'll talk about this in class).  We can also set the length of the Fast Fourier Transform (FFT) used with `nfft=`.  We can also chose the scaling (the units used for the y-axis, the 'power spectral density').  It is often good practice to detrend series before spectral analysis, and this can be done automatically with the `detrend=` command (see [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.detrend.html#scipy.signal.detrend) for options).  We're going to calculate the periodogram for both the cycle itself and the cycle+noise time series we created so we can compare them:

In [7]:

# Compute the periodogram including a Hamming window - you can select other windows if you'd like to see the differences: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.get_window.html#scipy.signal.get_window
window = "hamming"
nfft = 2**math.ceil(math.log2(np.size(my_series)))

# calculate the periodogram for both the known cycle and the cycle+noise case
f, Pxx = periodogram(my_series, window=window, nfft=nfft,scaling='spectrum')
f0, Pxx0 = periodogram(st, window=window, nfft=nfft,scaling='spectrum')


Scipy returns a value at the 0th frequency, which is difficult to properly interpret and will give a divide-by-zero error when we convert from frequency units (cycles per time) to period units (time per cycle). We'll remove the 0th frequency and the associated power sprectral density estimate:

In [8]:
# Avoid division by zero by removing first values from frequency and the spectrum
f = f[1:]  # remove zero frequency
Pxx = Pxx[1:] # remove corresponding power spectral density estimate
Pxx0 = Pxx0[1:] # remove corresponding power spectral density estimate


It can often be easier to think about periods (or wavelengths, the number of years long a cycle is, for instance) than frequency (the number of cycles per time), and this is a simple conversion:

In [9]:
# Convert frequency to period (from cycles/year to years/cycle) 
periods = 1 / f  # divide 1 by the frequency array to get a period array


finally, let's plot our spectra.   I'm highlight a few of the periods you might expect to see for different `cycles_per_year` from the code above (e.g. 1, 2, 3, 4, or 5):

In [ ]:

# Plot the periodogram with period (the years per cycle) on the x-axis
plt.figure(figsize=(10, 6))
plt.plot(periods, Pxx,'r',label="Cycle + Noise")  # Exclude the first point to avoid the division by zero problem (zero-frequency)
plt.plot(periods, Pxx0,'b',label="Cycle Only")  # Exclude the first point to avoid the division by zero problem (zero-frequency)

# Create some lines and labels the specific periods of interest given our 128 year time period
important_periods = [128, 64, 42, 32, 26] # various 'cycles per time' possibilities matching the above
for period in important_periods:
    plt.axvline(x=period, color='lightblue', linestyle='--', alpha=0.75)
    plt.text(period, max(Pxx)*0.8, f'{period} years', color='lightblue', rotation=-75, verticalalignment='center')

plt.xscale('log') # in this case, you could also use 'linear' here if you wanted
plt.xlabel('Period (Years)')
plt.ylabel('Power Spectral Density')
plt.legend()
plt.show()


You can make a few observations:  First, the blue curve showing the periodogram of the cycle alone has a sharp peak at the period corresponding to the 'years per cycle' determined by how many cycles were completed during the 1128 'years' of the time series.  The blue curve is also zero for most of the other periods.  But note that while the peak is sharp at the expected period, it isn't zero on either side of that peak -- there are 'lobes' of spurious variance that have leaked out of the true period.  This is one of the limitations of the periodogram but also generally a challenge to spectral analysis, and we'll talk about this in lecture. 

The red curve with the cycle and noise combined as some different features.  There is still a peak at the appropriate period and some leakage to either side, but now the signal peak is smaller (has less variance or power density).  Also because of the presense of the noise, the shorter end of the spectrum also shows various peaks ... peaks in the spectrum which are NOT reflective of a cyclical component.  Thus when dealing with real time series, be wary -- it is possible that many spurious peaks can be observed in a periodogram simply because of the presense of Gaussian noise.  

Take some time and play with the curve generation, the cycle lengths, and the noise and signal magnitudes, and then we'll move onto another time series and some additional methods we can use. 

# Sierra Nevada Precipitation 

We're going to be looking at data from [Williams et al. 2021](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2020WR028599). In this paper, we used observational and tree-ring reconstruction the Standardized Precipition Index (SPI) for the Sierra Nevada to investigate whether there were regular cycles in California winter precipitation that might provide predictability.  We'll mostly focus here on the observed November through March SPI data, which cover the period from 1902 to 2020. 

Let's read the data into a DataFrame and then plot it:

In [ ]:
filename = 'SPI_recon_NDJFM_SierraNevada.csv'

# read in the CSV file and make the 0th column (the years) the index
df = pd.read_csv(filename,index_col=0) 

# separate the DataFrame into separate Series 
reconstructed = df['reconstructed'].dropna() # split out the reconstructed values, dropping NaNs
observed = df['observed'].dropna() # split out the observed values, dropping NaNs

plt.plot(reconstructed,'k')
plt.plot(observed,'r')
plt.xlabel('Year')
plt.ylabel('SPI')
plt.title('SIERRA NEVADA WINTER SPI (Williams et al. 2021)')

plt.show()


We'll use the periodogram first to look for possible cyclical behavior in winter SPI in the Sierra Nevada observational record.  Once again, we'll pad FFT length to the next power of and using the Hanning window from above. We'll then plot the spectrum and highlight two of the potential oscillatory modes we focused on in the paper:

In [ ]:
# calculate the next power of 2 to pass to the periodogram for padding
nfft = 2**math.ceil(math.log2(np.size(observed)))

# calculate the simple periodogram 
f, Pxx = periodogram(observed, window=window, nfft=nfft,scaling='spectrum')

# there will be power at the 0 frequency, which is hard to interpret and creates problems converting to periods, so remove it
f = f[1:]  
Pxx = Pxx[1:]        

# Convert frequency to period  
periods = 1 / f  # Divide 1 by the frequency vector to get a period vector
important_periods = [2.2, 14] # a couple 'cycles per time' from Williams et al. 2021 we'll use to plot

# Plot the periodogram with period (years) on the x-axis
plt.figure(figsize=(8, 6))
plt.plot(periods, Pxx,'r',label="Cycle + Noise")  
plt.xscale("log")

# make the x-axis in 'normal' units (not scientific notation)
plt.gca().xaxis.set_major_formatter(ScalarFormatter())  # Use scalar formatter to allow us to avoid scientific notation
plt.gca().ticklabel_format(style='plain', axis='x')  

custom_ticks = [2, 2.5, 3.3, 5, 10, 20, 50, 100]  # Periods to label
plt.xticks(custom_ticks)
plt.xlim([1.9,110])
plt.xlabel('Period (years/cycle)')
plt.ylabel('Power Spectral Density')

# mark the important periods from Williams et al. 2021
for period in important_periods:
    plt.axvline(x=period, color='lightblue', linestyle='--', alpha=0.75)
    plt.text(period, max(Pxx)*0.8, f'{period} years', color='lightblue', rotation=-45, verticalalignment='center')

You can see that there are two clear peaks at ~2.2 years and between 13 and 15 years.   However, you'll also see that the spectrum is quite noisy, with multiple smaller peaks between 4 and 6 years. 

One thing you can do with periodogram spectra is smooth them directly.  This is quite often done with a Daniell filter, which is very similar to a moving average.  You can specify the span of the filter (the width) and the smoothing is accomplished by convolution of the filter (or kernel) and the time series:

In [ ]:
# Apply Daniell filter (moving average) for additional smoothing
def daniell_filter(data, width):
    kernel = np.ones(width) / width  # build the moving average kernel
    return np.convolve(data, kernel, mode='same') # convolve the filter

Pxx_smoothed = daniell_filter(Pxx,3)
Pxx_smoothed_more = daniell_filter(Pxx,5)

# Plot the periodogram with period (years) on the x-axis
plt.figure(figsize=(8, 6))
plt.scatter(periods, Pxx, color='pink', label="Periodogram", marker='o')  # Plot dots for the pink line
plt.plot(periods, Pxx_smoothed, color='red', label="Periodogram + Daniell Smoothing (3)")  # Red line for the smoothed periodogram
plt.plot(periods, Pxx_smoothed_more, color='darkred', label="Periodogram + Daniell Smoothing (5)")  # Red line for the smoothed periodogram
plt.legend()

plt.xscale("log")

# make the x-axis in normal units
plt.gca().xaxis.set_major_formatter(ScalarFormatter())  # Use scalar formatter to avoid scientific notation
plt.gca().ticklabel_format(style='plain', axis='x')  

custom_ticks = [2, 2.5, 3.3, 5, 10, 20, 50, 100]  # Example periods to label
plt.xticks(custom_ticks)
plt.xlim([1.9,110])
plt.xlabel('Period (years/cycle)')
plt.ylabel('Power Spectral Density')

for period in important_periods:
    plt.axvline(x=period, color='lightblue', linestyle='--', alpha=0.75)
    plt.text(period, max(Pxx)*0.8, f'{period} years', color='lightblue', rotation=-45, verticalalignment='center')

The choice of smoothing is somewhat arbitrary and depending on application, desired spectral resolution (or bandwidth), and a priori knowledge of your system.  You can see that the Daniell filter applied to the noisy periodogram spectrum of the observed Sierra Nevada SPI smooths out the features near 5 years. 

# Smoothed Periodogram from scratch

An alternative approach to spectral analysis is the Blackman-Tukey smoothed periodogram.  The primary benefit these days is that it gives you a smoother spectrum (but for a different reason than the Daniell post hoc filtering).  The Blackman-Tukey method starts from the autocovariance of the time signal itself, applying a window (often a Hann or Tukey window) to the autocovariance function.  The Discrete Fourier Transform (DFT) is then perform on that windowed autocorrelation function.  The result is a smoother version of the periodogram with reduced variance compared to that calculated directly, because the DFT is performed on the windowed autocovariance of the time series.  This comes at the expense of the spectral resolution, but can be desireable for reducing the influence of potential random fluctuations. 

There is no built-in function of the Blackman-Tukey smoothed periodogram in Python that I am aware of.  Here, I've attempted to build a function based on how MATLAB [does this](https://www.mathworks.com/help/ident/ref/spa.html).  See [here](https://www.mit.bme.hu/eng/system/files/oktatas/targyak/9132/Ljung_L_System_Identification_Theory_for_User-ed2.pdf) as well.  It is very possible this is not the most efficient or Pythonic way to do this, but I can reproduce fairly closely the results I get in MATLAB with the Python code I've written below, so this will do for now:


In [14]:
from numpy.fft import fft, ifft  # we'll use Numpy's fft in the function

def autocovariance(data, M):
   # Calculate the autocovariance values for lags 0 to M-1, this is directly mimicking MATLAB's covf function

    n = len(data)
    mean = np.mean(data)
    autocov = np.zeros(M) # pre-allocate return array
    
    for lag in range(M): # mimic how MATLAB does it, although probably more efficient ways to do this! 
        autocov[lag] = np.dot(data[:n-lag] - mean, data[lag:] - mean) / n
    
    return autocov

def hann_window(M):
    # This mimics how MATLAB does it, which is slightly different that Scipy's Hann window
    return 0.5 * (1 + np.cos(np.pi * np.arange(M) / (M - 1)))

def blackman_tukey_spectrum(data, M, fs=1):
    #  This performs the core of the Blackman-Tukey spectral estimation.

    # First, call the function above to calculate the lagged autocovariance 
    R = autocovariance(data, M)
    
    # Now, create and apply the Hann window to the autocovariance array, using the function above
    window = hann_window(M)
    R_windowed = R * window
    
    # calculate the next-power-of-2 and pad the windowed autocorrelation array
    nfft = 2 ** int(np.ceil(np.log2(2 * M)))  # next power of 2 
    R_padded = np.concatenate([R_windowed, np.zeros(nfft - M)]) # add the padding to the end
    
    # compute the FFT of the windowed autocovariance, and keep the real and positive parts only 
    spectrum_full = np.real(fft(R_padded))  
    spectrum = spectrum_full[:nfft // 2]
    
    # get the corresponding frequency values
    freqs = np.fft.fftfreq(nfft, fs)[:nfft // 2]
    
    return spectrum, freqs

Let's apply our new Blackman-Tukey function to the observed SPI data again.  One of the most important parameters is the window lag, $M$, and the choice will (as with many things in spectral analysis) reflect a balance between spectral resolution and variance.  A smaller $M$ will give a smoother spectrum with smaller variance, while a larger $M$ will reveal more and narrowed spectral peaks but generally a noisier periodogram.  The rule of thumb I learned was that $M$ should ideally fall between 1/3rd to 1/20th of the time series length, but other lengths are allowed. 

We'll start with $M=40$, but I encourage you to experiment:

In [ ]:
N = len(observed)
M = 40
fs=1

# call my Blackman-Tukey function above
Pxxbt, fbt = blackman_tukey_spectrum(observed, M, fs)
Pxxbt, fbt = Pxxbt[1:], fbt[1:]

# figure with 2 panel to compare the two spectra
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# First subplot
ax1.plot(1/f, Pxx, 'r')  # Plot for the first periodogram
ax1.set_xscale('log')
ax1.set_xlabel('Period (years/cycle)')
ax1.set_ylabel('Power Spectral Density')
ax1.xaxis.set_major_formatter(ScalarFormatter())  # Use scalar formatter for x-axis
ax1.ticklabel_format(style='plain', axis='x')  # Ensure no scientific notation
ax1.set_xticks([2, 2.5, 3.3, 5, 10, 20, 50, 100])  # Custom ticks
ax1.grid("both")
ax1.set_title('Periodogram')
for period in important_periods:
    ax1.axvline(x=period, color='lightblue', linestyle='--', alpha=0.75)
    ax1.text(period, max(Pxx)*0.8, f'{period} years', color='lightblue', rotation=-45, verticalalignment='center')
    
# Second subplot
ax2.plot(1/fbt, Pxxbt, 'r')  # Plot for the second periodogram
ax2.set_xscale('log')
ax2.set_xlabel('Period (years/cycle)')
ax2.set_ylabel('Power Spectral Density')
ax2.xaxis.set_major_formatter(ScalarFormatter())  # Use scalar formatter for x-axis
ax2.ticklabel_format(style='plain', axis='x')  # Ensure no scientific notation
ax2.set_xticks([2, 2.5, 3.3, 5, 10, 20, 50, 100])  # Custom ticks
ax2.grid("both")
ax2.set_title('Blackman-Tukey Smoothed Periodogram')
for period in important_periods:
    ax2.axvline(x=period, color='lightblue', linestyle='--', alpha=0.90)
    ax2.text(period, max(Pxxbt)*0.8, f'{period} years', color='lightblue', rotation=-45, verticalalignment='center')
    
# Adjust layout and show the plots
plt.tight_layout()
plt.show()


Make some observations about the Blackman-Tukey spectra as a function of different $M$.  How might the use of this particular approach to spectral analysis influence your investigation into cyclicity in California winter precipitation over the last several centuries? 

## Next steps

1. Apply spectral analysis to the reconstructed SPI series - how does it differ from the observed?  How do different choices, especially with respect to the Blackman-Tukey method and use of the Daniell filter affect your interpretation of the spectra? 
2. Explore additional spectral analysis tools - specifcally, please look at and try to use the [Welch method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.welch.html#scipy.signal.welch)
3. Why might you need to use the [Lomb-Scargle method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lombscargle.html#scipy.signal.lombscargle).  In general, what is one of the major assumptions of the type of time series analysis we've done thus far? 